In [ ]:
pip install lightfm

# Загрузка данных (признаки объектов)

In [ ]:
import pandas as pd
import numpy as np
from scipy.special import expit
from sklearn.metrics import roc_auc_score
from scipy.sparse import vstack, hstack, identity, csr_matrix
from lightfm.datasets import fetch_movielens
from scipy.io import mmwrite

movielens = fetch_movielens()
train = movielens['train'].tocsr()
test = movielens['test'].tocsr()

n_users = train.shape[0]
n_item_features = movielens['item_features'].shape[1]
d = n_users + n_item_features

user_features = identity(n_users, format='csr')
movie_features = movielens['item_features']
rows, cols = train.nonzero()
ratings = train.data
y = (ratings >= 4).astype(np.int32)

X = vstack([hstack([user_features[user_id], movie_features[movie_id]]).tocsr()
            for user_id, movie_id in zip(rows, cols)])

test_rows, test_cols = test.nonzero()
test_ratings = test.data
y1 = (test_ratings >= 4).astype(np.int32)

X1 = vstack([hstack([user_features[user_id], movie_features[movie_id]]).tocsr()
             for user_id, movie_id in zip(test_rows, test_cols)])

mmwrite("X_it.mm", X)
mmwrite("X1_it.mm", X1)
np.savetxt("y_it.csv", y, delimiter=",")
np.savetxt("y1_it.csv", y1, delimiter=",")

## EM-алгоритм

In [ ]:
r = 3

k = y - 0.5
iteration = 50
mu_0 = 0.0
mu_w_0 = 2.0
lambda_w_0 = 3.0
gamma_0 = 1.0
alpha_lambda = 1.0
beta_lambda = 1.0
eps = 1e-6

lambda_w = np.random.gamma(alpha_lambda, 1/beta_lambda)
mu_w = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_w)))
lambda_v = np.random.gamma(alpha_lambda, 1/beta_lambda, r)
mu_v = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_v)), r)

Theta_0 = 0.0
Theta_w = np.zeros(d)
Theta_v = np.zeros(d * r)
Theta_v_matrix = Theta_v.reshape((d, r))

X_squared = X.multiply(X)
model = Theta_0 + X @ Theta_w + 0.5 * np.sum((X @ Theta_v_matrix) ** 2 - X_squared @ Theta_v_matrix ** 2 , axis=1)
SaveResults = np.zeros((iteration, d + 1 + d * r))

for i in range(iteration):
    C = -0.5 * (1 / (2 * (model + eps)) * np.tanh((model + eps) / 2))
    model_iteration = Theta_0 + X @ Theta_w + 0.5 * np.sum((X @ Theta_v_matrix) ** 2 - X_squared @ Theta_v_matrix ** 2 , axis=1)

    A_w0 = np.sum(C) - lambda_w_0 / 2
    B_w0 = np.sum(k + 2 * C * (model_iteration - Theta_0)) + lambda_w_0 * mu_w_0
    if A_w0 < 0:
        Theta_0_new = -B_w0 / (2 * A_w0)
        model_iteration += (Theta_0_new - Theta_0)
        Theta_0 = Theta_0_new

    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        A_wj = np.sum(C * X_j**2) - lambda_w / 2
        B_wj = np.sum(k * X_j + 2 * C * (model_iteration - Theta_w[j] * X_j) * X_j) + lambda_w * mu_w
        if A_wj < 0:
            Theta_w_new = -B_wj / (2 * A_wj)
            model_iteration += (Theta_w_new - Theta_w[j]) * X_j
            Theta_w[j] = Theta_w_new

    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        for f in range(r):
            h_v = X_j * (X @ Theta_v_matrix[:, f]) - Theta_v_matrix[j, f] * X_j**2
            A_v = np.sum(C * h_v**2) - lambda_v[f] / 2
            B_v = np.sum(k * h_v + 2 * C * (model_iteration - Theta_v_matrix[j, f] * h_v) * h_v) + lambda_v[f] * mu_v[f]
            if A_v < 0:
                Theta_v_new = -B_v / (2 * A_v)
                model_iteration += (Theta_v_new - Theta_v_matrix[j, f]) * h_v
                Theta_v_matrix[j, f] = Theta_v_new

    model = model_iteration
    Theta = np.concatenate([[Theta_0], Theta_w, Theta_v_matrix.flatten()])
    SaveResults[i, :] = Theta

Theta_final = SaveResults[-1, :]
V = Theta_final[(d + 1):].reshape((d, r), order='C')

X1_squared = X1.multiply(X1)
double_sum = 0.5 * (np.sum((X1 @ V)**2, axis=1) - (X1_squared @ np.sum(V**2, axis=1)))
prob = expit(Theta_final[0] + X1 @ np.array(Theta_final[1:(d + 1)]) + double_sum)

auc_roc = roc_auc_score(y1, prob)
print("AUC-ROC на тестовой выборке:", auc_roc)

AUC-ROC на тестовой выборке: 0.7478364317223668


## EM-алгоритм ADAM

In [ ]:
r = 3
iteration = 50

# Фиксированные гиперпараметры
mu_0 = 0.0
mu_w_0 = 2.0
lambda_w_0 = 3.0
gamma_0 = 1.0
alpha_lambda = 1.0
beta_lambda = 1.0

alpha = 0.001
beta_1 = 0.9
beta_2 = 0.999
epsilon = 1e-8

eps = 1e-6
SaveResults = np.zeros((iteration, d + 1 + d * r))

lambda_w = np.random.gamma(alpha_lambda, 1/beta_lambda)
mu_w = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_w)))

lambda_v = np.random.gamma(alpha_lambda, 1/beta_lambda, r)
mu_v = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_v)), r)

k = y - 1 / 2

Theta_0 = 0
Theta_w = np.zeros(d)
Theta_v = np.zeros(d*r)
Theta_v_matrix = Theta_v.reshape((d, r))
Theta = np.concatenate((np.array([Theta_0]), Theta_w, Theta_v))

m_t = np.zeros_like(Theta)
v_t = np.zeros_like(Theta)
t = 0
X_squared = X.multiply(X)
for i in range(iteration):
    # E-шаг
    model = Theta_0 + X @ Theta_w + 0.5 * np.sum((X @ Theta_v_matrix) ** 2 - X_squared @ Theta_v_matrix ** 2 , axis=1)
    C = -0.5 * (1 / (2 * (model + eps)) * np.tanh((model + eps) / 2))

    grad = np.zeros_like(Theta)
    # Градиент для w_0
    grad[0] = np.sum(k + 2 * C * model) - lambda_w_0 * (Theta_0 - mu_w_0)

    # Градиент для w_j
    for j in range(d):
        grad[j + 1] = np.sum(k * X[:, j] + 2 * C * model * X[:, j]) - lambda_w * (Theta_w[j] - mu_w)

    # Градиент для v_jf
    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        for f in range(r):
            h_v = X_j * (X @ Theta_v_matrix[:, f]) - Theta_v_matrix[j, f] * X_j**2
            grad_idx = 1 + d + j * r + f
            grad[grad_idx] = np.sum(k * h_v + 2 * C * model * h_v) - lambda_v[f] * (Theta_v_matrix[j, f] - mu_v[f])

    t += 1
    m_t = beta_1 * m_t + (1 - beta_1) * grad
    v_t = beta_2 * v_t + (1 - beta_2) * grad**2
    m_hat = m_t / (1 - beta_1**t)
    v_hat = v_t / (1 - beta_2**t)
    Theta += alpha * m_hat / (np.sqrt(v_hat) + epsilon)

    Theta_0 = Theta[0]
    Theta_w = Theta[1:d+1]
    Theta_v_matrix = Theta[d+1:].reshape((d, r))
    SaveResults[i, :] = Theta

Theta_final = SaveResults[-1, :]
V = Theta_final[(d + 1):].reshape((d, r), order='C')

X1_squared = X1.multiply(X1)
double_sum = 0.5 * (np.sum((X1 @ V)**2, axis=1) - (X1_squared @ np.sum(V**2, axis=1)))
prob = expit(Theta_final[0] + X1 @ np.array(Theta_final[1:(d + 1)]) + double_sum)

# AUC-ROC
auc_roc = roc_auc_score(y1, prob)
print("AUC-ROC:", auc_roc)


AUC-ROC: 0.7284910672991085


## VFM ADAM

In [ ]:
r = 3
iterations = 50

mu_0 = 0.0
mu_w_0 = 2.0
lambda_w_0 = 3.0
gamma_0 = 1.0
alpha_lambda = 1.0
beta_lambda = 1.0

lambda_w = np.random.gamma(alpha_lambda, 1/beta_lambda)
mu_w = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_w)))
lambda_v = np.random.gamma(alpha_lambda, 1/beta_lambda, r)
mu_v = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_v)), r)

mu_w0_new = 0.0
sigma_w0_new = 0.1
mu_w_new = np.zeros(d)
sigma_w_new = np.ones(d)
mu_v_new = np.zeros((d, r))
sigma_v_new = np.ones((d, r))

m_mu_w0, v_mu_w0 = 0.0, 0.0
m_sigma_w0, v_sigma_w0 = 0.0, 0.0
m_mu_w, v_mu_w = np.zeros(d), np.zeros(d)
m_sigma_w, v_sigma_w = np.zeros(d), np.zeros(d)
m_mu_v, v_mu_v = np.zeros((d, r)), np.zeros((d, r))
m_sigma_v, v_sigma_v = np.zeros((d, r)), np.zeros((d, r))

mu_w0_all, mu_w_all, mu_v_all = [], [], []
alpha_k = np.ones(d)

def adam_update(param, grad, m, v, t, alpha=0.1, beta1=0.9, beta2=0.999, epsilon=1e-8):
    m_new = beta1 * m + (1 - beta1) * grad
    v_new = beta2 * v + (1 - beta2) * (grad ** 2)
    m_hat = m_new / (1 - beta1 ** t)
    v_hat = v_new / (1 - beta2 ** t)
    param += alpha * m_hat / (np.sqrt(v_hat) + epsilon)
    return param, m_new, v_new

for t in range(1, iterations + 1):
    epsilon_w0 = np.random.normal(0, 1)
    epsilon_w = np.random.normal(0, 1, d)
    epsilon_v = np.random.normal(0, 1, (d, r))

    Theta_0 = mu_w0_new + epsilon_w0 * sigma_w0_new
    Theta_w = mu_w_new + epsilon_w * sigma_w_new
    Theta_v = mu_v_new + epsilon_v * sigma_v_new

    X_squared = X.multiply(X)
    double_sum = 0.5 * (np.sum((X @ Theta_v) ** 2, axis=1) - X_squared @ np.sum(Theta_v ** 2, axis=1))

    grad_mu_w0 = np.sum(y - expit(Theta_0 + X @ Theta_w + double_sum)) - (d / np.sum(alpha_k)) * np.sum(alpha_k * (mu_w0_new - mu_w_0) / (1/lambda_w_0))
    grad_sigma_w0 = np.sum((y - expit(Theta_0 + X @ Theta_w + double_sum)) * epsilon_w0) - (d / np.sum(alpha_k)) * np.sum(alpha_k * (sigma_w0_new / (1/lambda_w_0) - 1 / sigma_w0_new))

    grad_mu_w = X.T @ (y - expit(Theta_0 + X @ Theta_w + double_sum))- (d / np.sum(alpha_k)) * (alpha_k * (mu_w_new - mu_w) / (1/lambda_w))

    grad_sigma_w = np.zeros(d)
    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        grad_sigma_w[j] = np.sum((y - expit(Theta_0 + X @ Theta_w + double_sum)) * X_j * epsilon_w[j]) - (d / np.sum(alpha_k)) * (alpha_k[j] * (sigma_w_new[j] / (1/lambda_w) - 1 / sigma_w_new[j]))

    grad_mu_v = np.zeros((d, r))
    grad_sigma_v = np.zeros((d, r))

    for f in range(r):
      h_v = X @ Theta_v[:, f] - X_squared @ Theta_v[:, f]
      grad_mu_v[:, f] = X.T @ ((y - expit(Theta_0 + X @ Theta_w + double_sum)) * h_v) - (d / np.sum(alpha_k)) * (alpha_k * (mu_v_new[:, f] - mu_v[f]) / (1/lambda_v[f]))
      for j in range(d):
          X_j = X[:, j].toarray().flatten()
          h_v_j = X_j * (X @ Theta_v[:, f]) - Theta_v[j, f] * X_j**2
          grad_sigma_v[j, f] = np.sum((y - expit(Theta_0 + X @ Theta_w + double_sum)) * h_v_j * epsilon_v[j, f]) - (d / np.sum(alpha_k)) * (alpha_k[j] * (sigma_v_new[j, f] / (1/lambda_v[f]) - 1 / sigma_v_new[j, f]))

    mu_w0_new, m_mu_w0, v_mu_w0 = adam_update(mu_w0_new, grad_mu_w0, m_mu_w0, v_mu_w0, t)
    sigma_w0_new, m_sigma_w0, v_sigma_w0 = adam_update(sigma_w0_new, grad_sigma_w0, m_sigma_w0, v_sigma_w0, t)
    mu_w_new, m_mu_w, v_mu_w = adam_update(mu_w_new, grad_mu_w, m_mu_w, v_mu_w, t)
    sigma_w_new, m_sigma_w, v_sigma_w = adam_update(sigma_w_new, grad_sigma_w, m_sigma_w, v_sigma_w, t)
    mu_v_new, m_mu_v, v_mu_v = adam_update(mu_v_new, grad_mu_v, m_mu_v, v_mu_v, t)
    sigma_v_new, m_sigma_v, v_sigma_v = adam_update(sigma_v_new, grad_sigma_v, m_sigma_v, v_sigma_v, t)

    mu_w0_all.append(mu_w0_new)
    mu_w_all.append(mu_w_new.copy())
    mu_v_all.append(mu_v_new.copy())

mu_w0_avg = np.mean(mu_w0_all, axis=0)
mu_w_avg = np.mean(mu_w_all, axis=0)
mu_v_avg = np.mean(mu_v_all, axis=0)

X1_squared = X1.multiply(X1)
double_sum_test = 0.5 * (np.sum((X1 @ mu_v_avg) ** 2, axis=1) - X1_squared @ np.sum(mu_v_avg ** 2, axis=1))
prob = expit(mu_w0_avg + X1 @ mu_w_avg + double_sum_test)

# AUC-ROC
auc_roc = roc_auc_score(y1, prob)
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.7357294048495966


# Загрузка данных (признаки объектов и пользователей)

In [ ]:
import pandas as pd
import numpy as np
from scipy.special import expit
from sklearn.metrics import roc_auc_score
from scipy.sparse import vstack, hstack, identity, csr_matrix
from lightfm.datasets import fetch_movielens
from scipy.io import mmwrite

movielens = fetch_movielens()
train = movielens['train'].tocsr()
test = movielens['test'].tocsr()
movie_features = movielens['item_features']

user_feature_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.user'
columns_user = ['userID', 'age', 'gender', 'occupation', 'zipcode']
user_data = pd.read_csv(user_feature_URL, sep='|', names=columns_user)

n_users = train.shape[0]
all_occupations = sorted(list(set(user_data['occupation'])))
occupation_matrix = np.zeros((len(user_data), len(all_occupations)))
for i, occ in enumerate(user_data['occupation']):
    occupation_matrix[i, all_occupations.index(occ)] = 1
user_features = hstack([identity(n_users, format='csr'), csr_matrix(occupation_matrix)])

n_item_features = movie_features.shape[1]
n_occupations = len(all_occupations)
d = n_users + n_item_features + n_occupations

rows, cols = train.nonzero()
ratings = train.data
y = (ratings >= 4).astype(np.int32)
X = vstack([hstack([user_features[user_id], movie_features[movie_id]]).tocsr()
            for user_id, movie_id in zip(rows, cols)])

test_rows, test_cols = test.nonzero()
test_ratings = test.data
y1 = (test_ratings >= 4).astype(np.int32)
X1 = vstack([hstack([user_features[user_id], movie_features[movie_id]]).tocsr()
             for user_id, movie_id in zip(test_rows, test_cols)])

mmwrite("X_it_u.mm", X)
mmwrite("X1_it_u.mm", X1)
np.savetxt("y_it_u.csv", y, delimiter=",")
np.savetxt("y1_it_u.csv", y1, delimiter=",")

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import vstack, hstack, identity, csr_matrix
from lightfm.datasets import fetch_movielens
from scipy.io import mmwrite

movielens = fetch_movielens()
train = movielens['train'].tocsr()
test = movielens['test'].tocsr()

user_feature_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.user'
columns_user = ['userID', 'age', 'gender', 'occupation', 'zipcode']
user_data = pd.read_csv(user_feature_URL, sep='|', names=columns_user)

n_users = train.shape[0]
all_occupations = sorted(list(set(user_data['occupation'])))
occupation_matrix = np.zeros((len(user_data), len(all_occupations)))
for i, occ in enumerate(user_data['occupation']):
    occupation_matrix[i, all_occupations.index(occ)] = 1
user_features = hstack([identity(n_users, format='csr'), csr_matrix(occupation_matrix)])  # (943, 964)

item_feature_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.item'
columns_item = ['movieID', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown',
                 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
                 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item_data = pd.read_csv(item_feature_URL, sep='|', names=columns_item, encoding='latin-1')
movie_genres = csr_matrix(item_data.iloc[:, 5:].values)

n_items = train.shape[1]
movie_ids = identity(n_items, format='csr')
movie_features = hstack([movie_ids, movie_genres])

n_item_features = movie_genres.shape[1]
n_occupations = len(all_occupations)
d = n_users + n_items + n_occupations + n_item_features

rows, cols = train.nonzero()
ratings = train.data
y = (ratings >= 4).astype(np.int32)
X = vstack([hstack([user_features[user_id], movie_features[movie_id]]).tocsr()
            for user_id, movie_id in zip(rows, cols)])

test_rows, test_cols = test.nonzero()
test_ratings = test.data
y1 = (test_ratings >= 4).astype(np.int32)
X1 = vstack([hstack([user_features[user_id], movie_features[movie_id]]).tocsr()
             for user_id, movie_id in zip(test_rows, test_cols)])

## EM-алгоритм

In [ ]:
r = 3
iteration = 50

mu_0 = 0.0
mu_w_0 = 2.0
lambda_w_0 = 3.0
gamma_0 = 1.0
alpha_lambda = 1.0
beta_lambda = 1.0
eps = 1e-6

lambda_w = np.random.gamma(alpha_lambda, 1/beta_lambda)
mu_w = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_w)))
lambda_v = np.random.gamma(alpha_lambda, 1/beta_lambda, r)
mu_v = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_v)), r)

Theta_0 = 0.0
Theta_w = np.zeros(d)
Theta_v = np.zeros(d * r)
Theta_v_matrix = Theta_v.reshape((d, r))

X_squared = X.multiply(X)
model = Theta_0 + X @ Theta_w + 0.5 * np.sum((X @ Theta_v_matrix) ** 2 - X_squared @ Theta_v_matrix ** 2 , axis=1)

SaveResults = np.zeros((iteration, d + 1 + d * r))

for i in range(iteration):
    C = -0.5 * (1 / (2 * (model + eps)) * np.tanh((model + eps) / 2))
    model_iteration = Theta_0 + X @ Theta_w + 0.5 * np.sum((X @ Theta_v_matrix) ** 2 - X_squared @ Theta_v_matrix ** 2 , axis=1)

    A_w0 = np.sum(C) - lambda_w_0 / 2
    B_w0 = np.sum(y - 0.5 + 2 * C * (model_iteration - Theta_0)) + lambda_w_0 * mu_w_0
    if A_w0 < 0:
        Theta_0_new = -B_w0 / (2 * A_w0)
        model_iteration += (Theta_0_new - Theta_0)
        Theta_0 = Theta_0_new

    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        A_wj = np.sum(C * X_j**2) - lambda_w / 2
        B_wj = np.sum((y - 0.5) * X_j + 2 * C * (model_iteration - Theta_w[j] * X_j) * X_j) + lambda_w * mu_w
        if A_wj < 0:
            Theta_w_new = -B_wj / (2 * A_wj)
            model_iteration += (Theta_w_new - Theta_w[j]) * X_j
            Theta_w[j] = Theta_w_new

    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        for f in range(r):
            h_v = X_j * (X @ Theta_v_matrix[:, f]) - Theta_v_matrix[j, f] * X_j**2
            A_v = np.sum(C * h_v**2) - lambda_v[f] / 2
            B_v = np.sum((y - 0.5) * h_v + 2 * C * (model_iteration - Theta_v_matrix[j, f] * h_v) * h_v) + lambda_v[f] * mu_v[f]
            if A_v < 0:
                Theta_v_new = -B_v / (2 * A_v)
                model_iteration += (Theta_v_new - Theta_v_matrix[j, f]) * h_v
                Theta_v_matrix[j, f] = Theta_v_new

    model = model_iteration
    Theta = np.concatenate([[Theta_0], Theta_w, Theta_v_matrix.flatten()])
    SaveResults[i, :] = Theta

Theta_final = SaveResults[-1, :]
V = Theta_final[(d + 1):].reshape((d, r), order='C')

X1_squared = X1.multiply(X1)

double_sum = 0.5 * (np.sum((X1 @ V)**2, axis=1) - (X1_squared @ np.sum(V**2, axis=1)))
prob = expit(Theta_final[0] + X1 @ np.array(Theta_final[1:(d + 1)]) + double_sum)

auc_roc = roc_auc_score(y1, prob)
print("AUC-ROC на тестовой выборке:", auc_roc)


AUC-ROC на тестовой выборке: 0.7577056959958239


## EM-алгоритм ADAM

In [ ]:
r = 3

iteration = 50

# Фиксированные гиперпараметры
mu_0 = 0.0
mu_w_0 = 2.0
lambda_w_0 = 3.0
gamma_0 = 1.0
alpha_lambda = 1.0
beta_lambda = 1.0

alpha = 0.001
beta_1 = 0.9
beta_2 = 0.999
epsilon = 1e-8

eps = 1e-6
SaveResults = np.zeros((iteration, d + 1 + d * r))

lambda_w = np.random.gamma(alpha_lambda, 1/beta_lambda)
mu_w = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_w)))

lambda_v = np.random.gamma(alpha_lambda, 1/beta_lambda, r)
mu_v = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_v)), r)

k = y - 1 / 2

Theta_0 = 0
Theta_w = np.zeros(d)
Theta_v = np.zeros(d*r)
Theta_v_matrix = Theta_v.reshape((d, r))
Theta = np.concatenate((np.array([Theta_0]), Theta_w, Theta_v))

m_t = np.zeros_like(Theta)
v_t = np.zeros_like(Theta)
t = 0
X_squared = X.multiply(X)
for i in range(iteration):
    # E-шаг
    model = Theta_0 + X @ Theta_w + 0.5 * np.sum((X @ Theta_v_matrix) ** 2 - X_squared @ Theta_v_matrix ** 2 , axis=1)
    C = -0.5 * (1 / (2 * (model + eps)) * np.tanh((model + eps) / 2))

    grad = np.zeros_like(Theta)
    # Градиент для w_0
    grad[0] = np.sum(k + 2 * C * model) - lambda_w_0 * (Theta_0 - mu_w_0)

    # Градиент для w_j
    for j in range(d):
        grad[j + 1] = np.sum(k * X[:, j] + 2 * C * model * X[:, j]) - lambda_w * (Theta_w[j] - mu_w)

    # Градиент для v_jf
    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        for f in range(r):
            h_v = X_j * (X @ Theta_v_matrix[:, f]) - Theta_v_matrix[j, f] * X_j**2
            grad_idx = 1 + d + j * r + f
            grad[grad_idx] = np.sum(k * h_v + 2 * C * model * h_v) - lambda_v[f] * (Theta_v_matrix[j, f] - mu_v[f])

    t += 1
    m_t = beta_1 * m_t + (1 - beta_1) * grad
    v_t = beta_2 * v_t + (1 - beta_2) * grad**2
    m_hat = m_t / (1 - beta_1**t)
    v_hat = v_t / (1 - beta_2**t)
    Theta += alpha * m_hat / (np.sqrt(v_hat) + epsilon)

    Theta_0 = Theta[0]
    Theta_w = Theta[1:d+1]
    Theta_v_matrix = Theta[d+1:].reshape((d, r))
    SaveResults[i, :] = Theta

Theta_final = SaveResults[-1, :]
V = Theta_final[(d + 1):].reshape((d, r), order='C')

X1_squared = X1.multiply(X1)
double_sum = 0.5 * (np.sum((X1 @ V)**2, axis=1) - (X1_squared @ np.sum(V**2, axis=1)))
prob = expit(Theta_final[0] + X1 @ np.array(Theta_final[1:(d + 1)]) + double_sum)

# AUC-ROC
auc_roc = roc_auc_score(y1, prob)
print("AUC-ROC:", auc_roc)


AUC-ROC: 0.7154805984791655


## VFM ADAM

In [ ]:
r = 3
iterations = 50

mu_0 = 0.0
mu_w_0 = 2.0
lambda_w_0 = 3.0
gamma_0 = 1.0
alpha_lambda = 1.0
beta_lambda = 1.0

lambda_w = np.random.gamma(alpha_lambda, 1/beta_lambda)
mu_w = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_w)))
lambda_v = np.random.gamma(alpha_lambda, 1/beta_lambda, r)
mu_v = np.random.normal(mu_0, np.sqrt(1/(gamma_0*lambda_v)), r)

mu_w0_new = 0.0
sigma_w0_new = 0.1
mu_w_new = np.zeros(d)
sigma_w_new = np.ones(d)
mu_v_new = np.zeros((d, r))
sigma_v_new = np.ones((d, r))

m_mu_w0, v_mu_w0 = 0.0, 0.0
m_sigma_w0, v_sigma_w0 = 0.0, 0.0
m_mu_w, v_mu_w = np.zeros(d), np.zeros(d)
m_sigma_w, v_sigma_w = np.zeros(d), np.zeros(d)
m_mu_v, v_mu_v = np.zeros((d, r)), np.zeros((d, r))
m_sigma_v, v_sigma_v = np.zeros((d, r)), np.zeros((d, r))

mu_w0_all, mu_w_all, mu_v_all = [], [], []
alpha_k = np.ones(d)

def adam_update(param, grad, m, v, t, alpha=0.1, beta1=0.9, beta2=0.999, epsilon=1e-8):
    m_new = beta1 * m + (1 - beta1) * grad
    v_new = beta2 * v + (1 - beta2) * (grad ** 2)
    m_hat = m_new / (1 - beta1 ** t)
    v_hat = v_new / (1 - beta2 ** t)
    param += alpha * m_hat / (np.sqrt(v_hat) + epsilon)
    return param, m_new, v_new

for t in range(1, iterations + 1):
    epsilon_w0 = np.random.normal(0, 1)
    epsilon_w = np.random.normal(0, 1, d)
    epsilon_v = np.random.normal(0, 1, (d, r))

    Theta_0 = mu_w0_new + epsilon_w0 * sigma_w0_new
    Theta_w = mu_w_new + epsilon_w * sigma_w_new
    Theta_v = mu_v_new + epsilon_v * sigma_v_new

    X_squared = X.multiply(X)
    double_sum = 0.5 * (np.sum((X @ Theta_v) ** 2, axis=1) - X_squared @ np.sum(Theta_v ** 2, axis=1))

    grad_mu_w0 = np.sum(y - expit(Theta_0 + X @ Theta_w + double_sum)) - (d / np.sum(alpha_k)) * np.sum(alpha_k * (mu_w0_new - mu_w_0) / (1/lambda_w_0))
    grad_sigma_w0 = np.sum((y - expit(Theta_0 + X @ Theta_w + double_sum)) * epsilon_w0) - (d / np.sum(alpha_k)) * np.sum(alpha_k * (sigma_w0_new / (1/lambda_w_0) - 1 / sigma_w0_new))

    grad_mu_w = X.T @ (y - expit(Theta_0 + X @ Theta_w + double_sum))- (d / np.sum(alpha_k)) * (alpha_k * (mu_w_new - mu_w) / (1/lambda_w))

    grad_sigma_w = np.zeros(d)
    for j in range(d):
        X_j = X[:, j].toarray().flatten()
        grad_sigma_w[j] = np.sum((y - expit(Theta_0 + X @ Theta_w + double_sum)) * X_j * epsilon_w[j]) - (d / np.sum(alpha_k)) * (alpha_k[j] * (sigma_w_new[j] / (1/lambda_w) - 1 / sigma_w_new[j]))

    grad_mu_v = np.zeros((d, r))
    grad_sigma_v = np.zeros((d, r))

    for f in range(r):
      h_v = X @ Theta_v[:, f] - X_squared @ Theta_v[:, f]
      grad_mu_v[:, f] = X.T @ ((y - expit(Theta_0 + X @ Theta_w + double_sum)) * h_v) - (d / np.sum(alpha_k)) * (alpha_k * (mu_v_new[:, f] - mu_v[f]) / (1/lambda_v[f]))
      for j in range(d):
          X_j = X[:, j].toarray().flatten()
          h_v_j = X_j * (X @ Theta_v[:, f]) - Theta_v[j, f] * X_j**2
          grad_sigma_v[j, f] = np.sum((y - expit(Theta_0 + X @ Theta_w + double_sum)) * h_v_j * epsilon_v[j, f]) - (d / np.sum(alpha_k)) * (alpha_k[j] * (sigma_v_new[j, f] / (1/lambda_v[f]) - 1 / sigma_v_new[j, f]))

    mu_w0_new, m_mu_w0, v_mu_w0 = adam_update(mu_w0_new, grad_mu_w0, m_mu_w0, v_mu_w0, t)
    sigma_w0_new, m_sigma_w0, v_sigma_w0 = adam_update(sigma_w0_new, grad_sigma_w0, m_sigma_w0, v_sigma_w0, t)
    mu_w_new, m_mu_w, v_mu_w = adam_update(mu_w_new, grad_mu_w, m_mu_w, v_mu_w, t)
    sigma_w_new, m_sigma_w, v_sigma_w = adam_update(sigma_w_new, grad_sigma_w, m_sigma_w, v_sigma_w, t)
    mu_v_new, m_mu_v, v_mu_v = adam_update(mu_v_new, grad_mu_v, m_mu_v, v_mu_v, t)
    sigma_v_new, m_sigma_v, v_sigma_v = adam_update(sigma_v_new, grad_sigma_v, m_sigma_v, v_sigma_v, t)

    mu_w0_all.append(mu_w0_new)
    mu_w_all.append(mu_w_new.copy())
    mu_v_all.append(mu_v_new.copy())


mu_w0_avg = np.mean(mu_w0_all, axis=0)
mu_w_avg = np.mean(mu_w_all, axis=0)
mu_v_avg = np.mean(mu_v_all, axis=0)

X1_squared = X1.multiply(X1)

double_sum = 0.5 * np.sum((X1 @ mu_v_avg)**2 - X1_squared @ (mu_v_avg**2), axis=1)
prob = expit(mu_w0_avg + X1 @ mu_w_avg + double_sum)

# AUC-ROC
auc_roc = roc_auc_score(y1, prob)
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.6995825406693135
